In [1]:
import numpy as np
import pandas as pd # for loading data

In [2]:
cd ..

/home/nicktehrany/Documents/Uni/3rd Year/P4/Machine Learning/Project


In [3]:
# Importing our own Functions
from util.helperfunctions import one_hot, one_hot_genres, cnn_clean_text, one_hot_reverse

/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/frame

In [4]:
cd src

/home/nicktehrany/Documents/Uni/3rd Year/P4/Machine Learning/Project/src


In [5]:
train_df = pd.read_csv("train.csv")

In [6]:
lyrics, max_len, word_list = cnn_clean_text(train_df['Lyrics'], 1)

# writing cleaned text back to df
index = 0
for text in lyrics:
    train_df['Lyrics'][index] = text
    index+=1
del lyrics

In [7]:
words = []
words.append('<PAD>')
for word in word_list:
    words.append(word)

In [8]:
train_lyrics = np.zeros(shape=(len(train_df), max_len))

row_index = 0
for row in train_df['Lyrics']:
    single_words = row.split()
    col_index = 0
    for word in single_words:
        train_lyrics[row_index][col_index] = words.index(word)
        col_index+=1
    row_index+=1

In [9]:
print(train_lyrics.shape)

(10098, 1525)


In [12]:
word2vec = {}
with open('w2v_cbow_model.word2vec', encoding='UTF-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec

In [16]:
EMBEDDING_DIM = 100
embedding_matrix = np.zeros((len(words), EMBEDDING_DIM))
index = 0
for word in words:
    embedding_vector = word2vec.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[index] = embedding_vector
    index+=1


In [30]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(len(words),
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=False)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [19]:
train_genres = one_hot_genres(train_df['Genre'])

In [20]:
val_df = pd.read_csv("validation.csv")
lyrics, dummy_len, dummy_list = cnn_clean_text(val_df['Lyrics'], 0)

# writing cleaned text back to df
index = 0
for text in lyrics:
    val_df['Lyrics'][index] = text
    index+=1
del lyrics

In [21]:
val_genres = one_hot_genres(val_df['Genre'])

In [23]:
val_lyrics = np.zeros(shape=(len(val_df), max_len))

row_index = 0
for row in val_df['Lyrics']:
    single_words = row.split()
    col_index = 0
    for word in single_words:
        if word in word_list:
            val_lyrics[row_index][col_index] = words.index(word)
        col_index+=1
    row_index+=1

In [24]:
test_df = pd.read_csv("test.csv")
lyrics, dummy_len, dummy_list = cnn_clean_text(test_df['Lyrics'], 0)

# writing cleaned text back to df
index = 0
for text in lyrics:
    test_df['Lyrics'][index] = text
    index+=1
del lyrics

In [25]:
test_genres = one_hot_genres(test_df['Genre'])

In [26]:
test_lyrics = np.zeros(shape=(len(test_df), max_len))

row_index = 0
for row in test_df['Lyrics']:
    single_words = row.split()
    col_index = 0
    for word in single_words:
        if word in word_list:
            test_lyrics[row_index][col_index] = words.index(word)
        col_index+=1
    row_index+=1

In [27]:
cnn_lyrics_train = train_lyrics[:, :, None]
cnn_lyrics_val = val_lyrics[:, :, None]
cnn_lyrics_test = test_lyrics[:, :, None]

In [28]:
print(cnn_lyrics_train.shape)

(10098, 1525, 1)


In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, MaxPool2D, Dropout, Flatten

model = Sequential()
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(16, activation='relu')) 
model.add(Dropout(0.2))
model.add(Dense(8, activation='softmax')) # Last layer: 10 class nodes, with dropout
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1525, 100)         3426100   
_________________________________________________________________
flatten_7 (Flatten)          (None, 152500)            0         
_________________________________________________________________
dense (Dense)                (None, 16)                2440016   
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 136       
Total params: 5,866,252
Trainable params: 2,440,152
Non-trainable params: 3,426,100
_________________________________________________________________


In [39]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_lyrics, train_genres, epochs=7, shuffle=True, batch_size=16, validation_data=(val_lyrics, val_genres))

Train on 10098 samples, validate on 2142 samples
Epoch 1/7
10098/10098 [==============================] - 6s 547us/sample - loss: 2.0758 - acc: 0.1364 - val_loss: 2.0572 - val_acc: 0.1293
Epoch 2/7
10098/10098 [==============================] - 5s 518us/sample - loss: 2.0364 - acc: 0.1714 - val_loss: 1.9128 - val_acc: 0.2418
Epoch 3/7
10098/10098 [==============================] - 5s 523us/sample - loss: 1.9310 - acc: 0.2239 - val_loss: 1.8861 - val_acc: 0.2535
Epoch 4/7
10098/10098 [==============================] - 5s 522us/sample - loss: 1.8403 - acc: 0.2478 - val_loss: 1.8379 - val_acc: 0.2834
Epoch 5/7
10098/10098 [==============================] - 5s 517us/sample - loss: 1.7853 - acc: 0.2704 - val_loss: 1.8342 - val_acc: 0.2680
Epoch 6/7
10098/10098 [==============================] - 5s 518us/sample - loss: 1.7211 - acc: 0.2912 - val_loss: 1.8330 - val_acc: 0.2838
Epoch 7/7
10098/10098 [==============================] - 5s 513us/sample - loss: 1.6740 - acc: 0.2981 - val_loss: 1.8

In [41]:
results = model.evaluate(test_lyrics, test_genres, batch_size=16)

2160/2160 [==============================] - 0s 158us/sample - loss: 1.8552 - acc: 0.2644
